In [2]:
import pandas as pd
import numpy as np
import sys

file_path = 'imdb_network.csv'

# df = pd.DataFrame(new_data, columns=column_names)
# df.set_index('tconst', inplace=True)
# df.to_csv('imdb_years_df.csv', sep=',')

In [3]:
# file headers
# tconst,nconst,primaryTitle,primaryName,primaryProfession
class TitleDictionary:

    def __init__(self, csv_path):
        self.df = pd.read_csv(csv_path)
        self.title_dict = self._create_title_dict()
        self.profession_dict = self._create_profession_dict()

    def _create_title_dict(self):
        title_dict = {}
        data = self.df.values.tolist()

        for entry in data:
            title = entry[2]
            if entry[1] in title_dict:
                title_dict[entry[1]].append(title)
            else:
                title_dict[entry[1]] = [title]

        # print(title_dict['nm6081065']) # debug
        return title_dict 

    def _create_profession_dict(self):
        profession_dict = {}
        data = self.df.values.tolist()

        for entry in data:
            if entry[1] not in profession_dict:
                profession = entry[3]
                if entry[4] == 'actor':
                    profession+='_a'
                else:
                    profession+='_d'

                profession_dict[entry[1]] = profession

        # print(profession_dict['nm6446679']) # debug
        return profession_dict

file_path = 'imdb_network.csv'
test = TitleDictionary(file_path)

In [9]:
td = TitleDictionary('imdb_network.csv')
nconst_title = td.title_dict
nconst_ar_dr = td.profession_dict

print(nconst_title["nm4655646"])

['White as Snow', 'Noose of Ice', 'Point 783', 'Seek and Destroy', 'The Launching']


In [11]:
#Graph Network Creation
'''
adj list
when adding edges
    if less than 2 movies in common, dont add the edge
    if actor->director, reverse the edge to director_actor
    if actor->actor, edge is bidirectional, so add the same edge for the actor

create graph is going to be a dict{dicts{}}, map of maps

add node
'''
class MovieNetwork:
    def __init__(self, name_movie_dict, nconst_ar_dr):
        self.graph = {} 
        self.name_movie_dict = name_movie_dict 
        self.nconst_ar_dr = nconst_ar_dr 
        
    def add_node(self, node):
        self.graph[node] = {}

    def edge_bi(self, node1, node2, weigh):
        if node1 not in self.graph: self.add_node(node1)
        if node2 not in self.graph: self.add_node(node2)

        if node2 not in self.graph[node1]:
            self.graph[node1][node2] = weigh

        if node1 not in self.graph[node2]:
            self.graph[node2][node1] = weigh

    def edge_single(self, node1, node2, weigh):
        if node1 not in self.graph: self.add_node(node1)
        if node2 not in self.graph: self.add_node(node2)
        
        if node2 not in self.graph[node1]:
            self.graph[node1][node2] = weigh

    # nconst_ar_dr, weight=1
    # old parameters, not needed
    def add_edge(self, node1, node2):
        node1_movies = self.name_movie_dict[node1]
        node2_movies = self.name_movie_dict[node2]

        empty_set = set()
        # checking movies in common
        for movie in node1_movies:
            if movie in node2_movies:
                # weight+=1
                empty_set.add(movie)
        
        weight = len(empty_set)
                
        # only proceed if weights are more than 2
        if weight > 2:
            node1_ar_dr = self.nconst_ar_dr[node1]
            node1_profession = node1_ar_dr[len(node1_ar_dr)-1:]

            node2_ar_dr = self.nconst_ar_dr[node2]
            node2_profession = node2_ar_dr[len(node2_ar_dr)-1:]

            if node1_profession == node2_profession:
                self.edge_bi(node1, node2, weight)
            if node1_profession == "a" and node2_profession == "d":
                self.edge_single(node2, node1, weight)
            if node1_profession == "d" and node2_profession == "a":
                self.edge_single(node1, node2, weight)

    def get_graph(self):
        for node1 in self.nconst_ar_dr:
            for node2 in self.nconst_ar_dr:
                if node1 == node2:
                    continue
                if node1 not in self.graph: self.add_node(node1)
                if node2 not in self.graph: self.add_node(node2)
                self.add_edge(node1, node2)
        
        return self.graph

In [12]:
td = TitleDictionary('imdb_network.csv')
nconst_title = td.title_dict
nconst_ar_dr = td.profession_dict

# print(nconst_title['nm1662883'])

movie_network = MovieNetwork(nconst_title, nconst_ar_dr)

graph = movie_network.get_graph()

In [13]:
print(graph['nm4655646'])

{'nm8706223': 5}


In [9]:
# testing code 
# print(18871 + 15753)

s = 'Benjamin H. Kline_d'

print(s[len(s)-1:])



# file_path = 'imdb_network.csv'
# data = pd.read_csv(file_path)
# data = data.values.tolist()

# print(len(data))

# for i in data:
#     print(i)

d
